# Import Optuna and Packages

---



In [0]:
try:
  !pip install git+https://github.com/optuna/optuna.git
  !pip install scikit-optimize
except:
  print('Problem')

  Cloning https://github.com/optuna/optuna.git to /tmp/pip-req-build-74ybvvw4
  Running command git clone -q https://github.com/optuna/optuna.git /tmp/pip-req-build-74ybvvw4
     |████████████████████████████████| 1.1MB 4.7MB/s 
     |████████████████████████████████| 81kB 5.0MB/s 
     |████████████████████████████████| 471kB 24.9MB/s 
     |████████████████████████████████| 61kB 12.0MB/s 
     |████████████████████████████████| 112kB 28.2MB/s 
     |████████████████████████████████| 51kB 9.3MB/s 
  Created wheel for optuna: filename=optuna-1.0.0-cp36-none-any.whl size=185569 sha256=62a6ea111ebb59fd92239bcc36a15f5ecbd4b95feb6f6b45d0b05c5532edd429
  Stored in directory: /tmp/pip-ephem-wheel-cache-ejmyldl7/wheels/7c/1e/3e/4c91637ddd97448a8daf4600bd12ddf42e1e38bdc28d255ef1
  Created wheel for alembic: filename=alembic-1.3.2-py2.py3-none-any.whl size=151128 sha256=58ff4d71415f8ac503df9f4837fc0799206d0395fccf46d5809e46eeb290425c
  Stored in directory: /root/.cache/pip/wheels/5c/66/53/e0633

In [0]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import pandas as pd
import random as rnd

import seaborn as sns
from matplotlib import style
style.use('seaborn-pastel')

import keras
from keras import backend as K
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, LeakyReLU, ReLU
from keras.models import Model, Sequential, load_model
from keras.utils import to_categorical
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,StratifiedKFold

import random
import pickle

import warnings
warnings.filterwarnings("ignore")

import optuna 
print('Optuna', optuna.__version__)

Using TensorFlow backend.


Optuna 1.0.0


In [0]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

## Prepare Data and ENV

---



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Collegamento al server sqlite per salvare automaticamente ogni trial dell'oggeto study di ottimizzazione

In [0]:
import sqlite3
from sqlite3 import Error

# /content/drive/My Drive/sqlite/db/opt.db --> LCB (RF)
# /content/drive/My Drive/sqlite/db/opt_ei_rf.db --> EI (RF)
 
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print('Version sqlite: {}'.format(sqlite3.version))
        print('Connected')
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()
 
 
if __name__ == '__main__':
    create_connection('/content/drive/My Drive/sqlite/db/optuna.db')

Version sqlite: 2.6.0
Connected


In [0]:
with np.load('/content/drive/My Drive/extracted_3fold1.npz') as data:
  # training set
  x_train1 = data['x_train1']
  y_train1 = data['y_train1']

  # validation set
  x_val1 = data['x_val1']
  y_val1 = data['y_val1']

  # test set
  x_test1 = data['x_test1']
  y_test1 = data['y_test1']

In [0]:
print('Shape del training set: {} --- {}\nShape del validation set: {} --- {}\nShape del test set: {} --- {}'. format(x_train1.shape, y_train1.shape,
                                                                                                                 x_val1.shape, y_val1.shape,
                                                                                                                 x_test1.shape, y_test1.shape))

Shape del training set: (5176, 14, 14, 1024) --- (5176,)
Shape del validation set: (432, 14, 14, 1024) --- (432,)
Shape del test set: (864, 14, 14, 1024) --- (864,)


In [0]:
with np.load('/content/drive/My Drive/extracted_3fold2.npz') as data:
  # training set
  x_train2 = data['x_train2']
  y_train2 = data['y_train2']

  # validation set
  x_val2 = data['x_val2']
  y_val2 = data['y_val2']

  # test set
  x_test2 = data['x_test2']
  y_test2 = data['y_test2']

In [0]:
print('Shape del training set: {} --- {}\nShape del validation set: {} --- {}\nShape del test set: {} --- {}'. format(x_train2.shape, y_train2.shape,
                                                                                                                 x_val2.shape, y_val2.shape,
                                                                                                                 x_test2.shape, y_test2.shape))

Shape del training set: (5180, 14, 14, 1024) --- (5180,)
Shape del validation set: (432, 14, 14, 1024) --- (432,)
Shape del test set: (863, 14, 14, 1024) --- (863,)


In [0]:
with np.load('/content/drive/My Drive/extracted_3fold3.npz') as data:
  # training set
  x_train3 = data['x_train3']
  y_train3 = data['y_train3']

  # validation set
  x_val3 = data['x_val3']
  y_val3 = data['y_val3']

  # test set
  x_test3 = data['x_test3']
  y_test3 = data['y_test3']

In [0]:
print('Shape del training set: {} --- {}\nShape del validation set: {} --- {}\nShape del test set: {} --- {}'. format(x_train3.shape, y_train3.shape,
                                                                                                                 x_val3.shape, y_val3.shape,
                                                                                                                 x_test3.shape, y_test3.shape))

Shape del training set: (5180, 14, 14, 1024) --- (5180,)
Shape del validation set: (432, 14, 14, 1024) --- (432,)
Shape del test set: (863, 14, 14, 1024) --- (863,)


In [0]:
le = LabelEncoder()

# y1
y_train1 = le.fit_transform(y_train1)
y_train1 = to_categorical(y_train1, num_classes = 7)
y_test1 = le.fit_transform(y_test1)
y_test1 = to_categorical(y_test1, num_classes = 7)
y_val1 = le.fit_transform(y_val1)
y_val1 = to_categorical(y_val1, num_classes = 7)

# y2
y_train2 = le.fit_transform(y_train2)
y_train2 = to_categorical(y_train2, num_classes = 7)
y_test2 = le.fit_transform(y_test2)
y_test2 = to_categorical(y_test2, num_classes = 7)
y_val2 = le.fit_transform(y_val2)
y_val2 = to_categorical(y_val2, num_classes = 7)

# y3
y_train3 = le.fit_transform(y_train3)
y_train3 = to_categorical(y_train3, num_classes = 7)
y_test3 = le.fit_transform(y_test3)
y_test3 = to_categorical(y_test3, num_classes = 7)
y_val3 = le.fit_transform(y_val3)
y_val3 = to_categorical(y_val3, num_classes = 7)

## Optimization with Optuna

---



In [0]:
filepath = "/content/drive/My Drive/model_opt/opt_best_{val_acc:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_acc', mode='max', patience=10, restore_best_weights=True)

# objective function 

def objective(trial):
  
  callbacks_list = [optuna.integration.KerasPruningCallback(trial, 'val_acc'), checkpoint, es]
  K.clear_session()

  accuracy_tmp = []
  # init params and callbacks
  batch_size = 128
  epochs = 30
  

  # params to optimize
  dropout_rate = trial.suggest_discrete_uniform('dropout_rate', 0, 1, .1)
  nlayer_d1 = trial.suggest_int('neurons_dense', 21, 490)
  activation = trial.suggest_categorical('activation', ['relu', 'leakyRelu'])
  optimizers = trial.suggest_categorical('optimizer', ['Adam', 'rmsprop', 'nadam'])
  lr = trial.suggest_loguniform('lr', 1e-6, 1e-2)
  filters = trial.suggest_int('filters', 32, 256)
  kernelSize = trial.suggest_int('size_filters', 1, 5)

  ######################################## fold 1 ########################################
  # Create model
  
  model = Sequential()
  if activation == 'relu':
    # C1
    model.add(Conv2D(filters, kernel_size=kernelSize, input_shape=(14,14,1024)))
    model.add(ReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(ReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(ReLU())
  else:
    # C1
    model.add(Conv2D(filters, kernel_size=kernelSize, input_shape=(14,14,1024)))
    model.add(LeakyReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(LeakyReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(LeakyReLU())  
  # flat
  model.add(Flatten())
  if activation == 'relu':
    # D1
    model.add(Dense(nlayer_d1,
                    kernel_regularizer=regularizers.l2(0.01)))
    model.add(ReLU())
  else:
    # D1
    model.add(Dense(nlayer_d1,
                    kernel_regularizer=regularizers.l2(0.01)))
    model.add(LeakyReLU())
  # dropout
  model.add(Dropout(dropout_rate))
  # output
  model.add(Dense(7, activation='softmax'))
  # optimizer
  if optimizers == 'Adam':
    OPTIMIZER = keras.optimizers.Adam(lr=lr)
  elif optimizers == 'nadam':
    OPTIMIZER = keras.optimizers.nadam(lr=lr)
  else:
    OPTIMIZER = keras.optimizers.rmsprop(lr=lr)
    # Compile model
  model.compile(loss = keras.losses.categorical_crossentropy,
                  optimizer = OPTIMIZER,
                  metrics = ['acc'])
  # fit model
  model.fit(x_train1, y_train1, 
            validation_data=(x_val1, y_val1),
            shuffle=True,
            batch_size = batch_size, epochs = epochs,
            callbacks=callbacks_list,
            verbose=0)
  # evaluate model
  #model.load_weights(filepath)
  scores = model.evaluate(x_test1, y_test1, verbose=0)
  accuracy_tmp.append(scores[1])

  K.clear_session()
  ######################################## fold 2 ########################################
  # Create model
  model = Sequential()
  if activation == 'relu':
    # C1
    model.add(Conv2D(filters, kernel_size=kernelSize, input_shape=(14,14,1024)))
    model.add(ReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(ReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(ReLU())
  else:
    # C1
    model.add(Conv2D(filters, kernel_size=kernelSize, input_shape=(14,14,1024)))
    model.add(LeakyReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(LeakyReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(LeakyReLU())   
  # flat
  model.add(Flatten())
  if activation == 'relu':
    # D1
    model.add(Dense(nlayer_d1,
                    kernel_regularizer=regularizers.l2(0.01)))
    model.add(ReLU())
  else:
    # D1
    model.add(Dense(nlayer_d1,
                    kernel_regularizer=regularizers.l2(0.01)))
    model.add(LeakyReLU())
  # dropout
  model.add(Dropout(dropout_rate))
  # output
  model.add(Dense(7, activation='softmax'))
  # optimizer
  if optimizers == 'Adam':
    OPTIMIZER = keras.optimizers.Adam(lr=lr)
  elif optimizers == 'nadam':
    OPTIMIZER = keras.optimizers.nadam(lr=lr)
  else:
    OPTIMIZER = keras.optimizers.rmsprop(lr=lr)
    # Compile model
  model.compile(loss = keras.losses.categorical_crossentropy,
                  optimizer = OPTIMIZER,
                  metrics = ['acc'])
  # fit model
  model.fit(x_train2, y_train2, 
            validation_data=(x_val2, y_val2),
            shuffle=True,
            batch_size = batch_size, epochs = epochs,
            callbacks=callbacks_list,
            verbose=0)
  # evaluate model
  #model.load_weights(filepath)
  scores = model.evaluate(x_test2, y_test2, verbose=0)
  accuracy_tmp.append(scores[1])

  K.clear_session()
  ######################################## fold 3 ########################################
  # Create model
  model = Sequential()
  if activation == 'relu':
    # C1
    model.add(Conv2D(filters, kernel_size=kernelSize, input_shape=(14,14,1024)))
    model.add(ReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(ReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(ReLU())
  else:
    # C1
    model.add(Conv2D(filters, kernel_size=kernelSize, input_shape=(14,14,1024)))
    model.add(LeakyReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(LeakyReLU())
    model.add(Conv2D(filters, kernel_size=kernelSize))
    model.add(LeakyReLU())  
  # flat
  model.add(Flatten())
  if activation == 'relu':
    # D1
    model.add(Dense(nlayer_d1,
                    kernel_regularizer=regularizers.l2(0.01)))
    model.add(ReLU())
  else:
    # D1
    model.add(Dense(nlayer_d1,
                    kernel_regularizer=regularizers.l2(0.01)))
  # dropout
  model.add(Dropout(dropout_rate))
  # output
  model.add(Dense(7, activation='softmax'))
  # optimizer
  if optimizers == 'Adam':
    OPTIMIZER = keras.optimizers.Adam(lr=lr)
  elif optimizers == 'nadam':
    OPTIMIZER = keras.optimizers.nadam(lr=lr)
  else:
    OPTIMIZER = keras.optimizers.rmsprop(lr=lr)
  # Compile model
  model.compile(loss = keras.losses.categorical_crossentropy,
                  optimizer = OPTIMIZER,
                  metrics = ['acc'])  
  # fit model
  model.fit(x_train3, y_train3, 
            validation_data=(x_val3, y_val3),
            shuffle=True,
            batch_size = batch_size, epochs = epochs,
            callbacks=callbacks_list,
            verbose=0)
  # evaluate model
  #model.load_weights(filepath)
  scores = model.evaluate(x_test3, y_test3, verbose=0)
  accuracy_tmp.append(scores[1])
  
  ######################################## VALUTATION FINAL ########################################
  accuracy = np.mean(accuracy_tmp)

  return (1-accuracy) # value to minimize


**RF - LCB**



In [0]:
# best in _Simple_CV_RF(LCB)_Checkpoint_EarlyStopping

# Load or create a new study
dataset_name = 'CNN'
study_name = dataset_name + '_Simple_CV_RF(LCB)_Checkpoint_EarlyStopping'

study = optuna.create_study(study_name=study_name,
                            direction='minimize',
                            sampler = optuna.integration.SkoptSampler(independent_sampler=optuna.samplers.TPESampler(),
                                                                      warn_independent_sampling = True,
                                                                      skopt_kwargs={'base_estimator':'RF', 'acq_func':'LCB'}),
                                   storage='sqlite:////content/drive/My Drive/sqlite/db/optuna.db', load_if_exists=True)

[I 2020-01-16 13:56:50,956] Using an existing study with name 'CNN_Simple_CV_RF(LCB)_Checkpoint_EarlyStopping' instead of creating a new one.


In [0]:
# optimize object study
study.optimize(objective, n_trials = 49) # n_trials or timeout (sec)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










[I 2020-01-16 14:09:32,962] Finished trial#54 resulted in value: 0.1023190173888181. Current best value is 0.04013820999745066 with parameters: {'activation': 'leakyRelu', 'dropout_rate': 0.5, 'filters': 191, 'lr': 1.364710328809509e-05, 'neurons_dense': 346, 'optimizer': 'Adam', 'size_filters': 4}.
[I 2020-01-16 14:09:42,276] Setting status of trial#55 as TrialState.PRUNED. Trial was pruned at epoch 1.
[I 2020-01-16 14:09:50,852] Setting status of trial#56 as TrialState.PRUNED. Trial was pruned at epoch 1.
[I 2020-01-16 14:09:58,619] Setting status of trial#57 as TrialState.PRUNED. Trial was pruned at epoch 1.
[I 2020-01-16 14:10:06,801] Setting status of trial#58 as TrialState.PRUNED. Trial was pruned at epoch 1.


[I 2020-01-16 14:14:24,832] Finished trial#59 resulted in value: 0.0961434956173538. Current best value is 0.04013820999745066 with parameters: {'activation': 'leakyRelu', 'dropout_rate': 0.5, 'filters': 191, 'lr': 1.364710328809509e-05, 'neurons_dense': 346, 'optimizer': 'Adam', 'size_filters': 4}.
[I 2020-01-16 14:17:58,516] Finished trial#60 resulted in value: 0.8428567979915025. Current best value is 0.04013820999745066 with parameters: {'activation': 'leakyRelu', 'dropout_rate': 0.5, 'filters': 191, 'lr': 1.364710328809509e-05, 'neurons_dense': 346, 'optimizer': 'Adam', 'size_filters': 4}.
[I 2020-01-16 14:19:47,765] Finished trial#61 resulted in value: 0.8513542873696408. Current best value is 0.04013820999745066 with parameters: {'activation': 'leakyRelu', 'dropout_rate': 0.5, 'filters': 191, 'lr': 1.364710328809509e-05, 'neurons_dense': 346, 'optimizer': 'Adam', 'size_filters': 4}.
[I 2020-01-16 14:23:11,627] Finished trial#62 resulted in value: 0.08609877259637877. Current bes

In [0]:
print('Best trial number: ', study.best_trial.number)
print('Best value:', 1-study.best_trial.value)
print('Best parameters: \n', study.best_trial.params)

Best trial number:  2
Best value: 0.9598617900025493
Best parameters: 
 {'activation': 'leakyRelu', 'dropout_rate': 0.5, 'filters': 191, 'lr': 1.364710328809509e-05, 'neurons_dense': 346, 'optimizer': 'Adam', 'size_filters': 4}


## Visualizations

---



In [0]:
study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))

In [0]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
print('Trail completed: {}'.format(len(df[study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))['state']=='COMPLETE'])))
print('Trail pruned: {}'.format(len(df[study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))['state']=='PRUNED'])))
print('Trail failed: {}'.format(len(df[study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))['state']=='FAIL'])))

Trail completed: 58
Trail pruned: 43
Trail failed: 2


In [0]:
optuna.visualization.plot_parallel_coordinate(study)

In [0]:
optuna.visualization.plot_contour(study, ['activation', 'neurons_dense', 'filters', 'size_filters'])

In [0]:
optuna.visualization.plot_optimization_history(study)

In [0]:
optuna.visualization.plot_intermediate_values(study)

In [0]:
optuna.visualization.plot_slice(study)